In [1]:
import requests
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib3
import requests_cache
from tqdm import tqdm
import langid
import numpy as np
import openai
from openai import OpenAI
import json
import hashlib

In [2]:
requests_cache.install_cache('.cache') 

In [3]:
# 🔇 Disable SSL warnings (for local, trusted scraping only)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
DATA = Path('~/Projects/MaliKAGF/Data')
CACHE_DIR = Path("chatgpt_cache")
CACHE_DIR.mkdir(exist_ok=True)

In [5]:
wfo_df = pd.read_csv(DATA / 'classification.csv', delimiter="\t", encoding="latin1")

/var/folders/32/mkdx8mcd3v7ff79wljjjv0d00000gp/T/ipykernel_94445/4257059906.py:1: DtypeWarning: Columns (2,8,9,10,12,14,28) have mixed types. Specify dtype option on import or set low_memory=False.
  wfo_df = pd.read_csv(DATA / 'classification.csv', delimiter="\t", encoding="latin1")


In [6]:
wfo_df.head()

,taxonID,scientificNameID,localID,scientificName,taxonRank,parentNameUsageID,scientificNameAuthorship,family,subfamily,tribe,...,acceptedNameUsageID,originalNameUsageID,nameAccordingToID,taxonRemarks,created,modified,references,source,majorGroup,tplID
0,wfo-0001302010,NaN,9905237,Schoenoxiphium ecklonii var. ecklonii,variety,NaN,NaN,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2023-05-29,NaN,The Cyperaceae TEN,A,tro-9905237
1,wfo-0001302011,NaN,9905253,Cyperus violifolia,species,NaN,Rodriguez & Alfonso,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2022-04-20,NaN,The Cyperaceae TEN,A,tro-9905253
2,wfo-0001302012,NaN,9905297,Carex viridula var. viridula,variety,NaN,NaN,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2023-05-29,NaN,The Cyperaceae TEN,A,tro-9905297
3,wfo-0001302013,urn:lsid:ipni.org:names:310980-1,9905332,Mariscus phleoides,species,NaN,Nees,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro Updated namePublished...,2022-04-16,2024-06-04,NaN,The Cyperaceae TEN,A,tro-9905332
4,wfo-0001302014,urn:lsid:ipni.org:names:315251-1,9905359,Tetraria compar,species,NaN,P.Beauv. ex T.Lestib.,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro Updated Author from L...,2022-04-16,2024-06-04,NaN,The Cyperaceae TEN,A,tro-9905359


In [7]:
gbif_df = pd.read_csv(DATA / 'gbif-mali-taxonomy.csv')

In [8]:
gbif_df.species

0       Phalaris arundinacea
1         Pandanus malgrasii
2       Platycoryne paludosa
3        Nervilia bicarinata
4           Nervilia petraea
                ...         
1531                     NaN
1532        Coccinia grandis
1533                     NaN
1534      Archidium ohioense
1535                     NaN
Name: species, Length: 1536, dtype: object

In [9]:
mali_wfo_df = wfo_df[wfo_df.scientificName.isin(gbif_df.species)]

In [10]:
def scrape_wfo_paragraphs(wfo_taxon_id):
    url = f"https://www.worldfloraonline.org/taxon/{wfo_taxon_id}"
    
    try:
        response = requests.get(url, verify=False, timeout=10)  # disable cert check
    except requests.exceptions.SSLError as e:
        print(f"SSL Error for {wfo_taxon_id}: {e}")
        return []
    except Exception as e:
        print(f"Request error for {wfo_taxon_id}: {e}")
        return []

    if response.status_code != 200:
        print(f"❌ Failed to retrieve page for {wfo_taxon_id}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    section_ids = ['general', 'morphology', 'habit', 'ecology', 'habitat', 'distribution', 'use']

    records = []
    for sec_id in section_ids:
        section_div = soup.find("div", id=sec_id)
        if section_div:
            desc_blocks = section_div.find_all("div", class_="description-with-citations")
            for block in desc_blocks:
                paragraphs = block.find_all("p", class_="justified")
                for para in paragraphs:
                    records.append({
                        "wfo_taxon_id": wfo_taxon_id,
                        "section": sec_id,
                        "text": para.get_text(strip=True)
                    })
    return records

In [11]:

results = []

for i, row in tqdm(mali_wfo_df.iterrows(), total=len(mali_wfo_df), desc="Processing WFO taxa"):
    name = row['scientificName']
    taxonID = row['taxonID']    
    result = scrape_wfo_paragraphs(taxonID)
    result = [{**d, 'name': name} for d in result]
    results.extend(result)
    

Processing WFO taxa:  91%|████████████████▍ | 1065/1169 [00:19<00:04, 25.95it/s]

❌ Failed to retrieve page for wfo-0001278471


Processing WFO taxa: 100%|██████████████████| 1169/1169 [00:20<00:00, 56.31it/s]


In [12]:
results_df = pd.DataFrame(results)

In [13]:
results_df

,wfo_taxon_id,section,text,name
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus
2,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii
3,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla
4,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza
...,...,...,...,...
5293,wfo-0001425157,habitat,Pans. It occurs along the sides of shallow sea...,Pontederia africana
5294,wfo-0001425157,distribution,The only known record for the region is from T...,Pontederia africana
5295,wfo-1000031633,morphology,"Herbeannuelle, de 10–100 cm de haut, aromatiqu...",Coleus rhodesianum
5296,wfo-1000031633,ecology,pionnière des sols perturbés ; au Gabon à 100–...,Coleus rhodesianum


In [14]:
def detect_langid(text):
    # Get top two language predictions and scores
    ranked = langid.rank(text)
    top_lang, top_score = ranked[0]
    second_lang, second_score = ranked[1]

    # Convert to softmax percentage confidence
    scores = np.array([top_score, second_score])
    probs = np.exp(scores - np.max(scores))  # for numerical stability
    probs = probs / probs.sum()

    return top_lang, float(probs[0]) * 100  # return as percentage

In [15]:
def apply_detect_langid(text):
    lang, conf = detect_langid(text)
    return pd.Series({'lang': lang, 'lang_confidence': conf})

results_df[['lang', 'lang_confidence']] = results_df['text'].apply(apply_detect_langid)

In [16]:
results_df

,wfo_taxon_id,section,text,name,lang,lang_confidence
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus,en,100.0
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus,en,100.0
2,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii,en,100.0
3,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla,es,100.0
4,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza,en,100.0
...,...,...,...,...,...,...
5293,wfo-0001425157,habitat,Pans. It occurs along the sides of shallow sea...,Pontederia africana,en,100.0
5294,wfo-0001425157,distribution,The only known record for the region is from T...,Pontederia africana,en,100.0
5295,wfo-1000031633,morphology,"Herbeannuelle, de 10–100 cm de haut, aromatiqu...",Coleus rhodesianum,fr,100.0
5296,wfo-1000031633,ecology,pionnière des sols perturbés ; au Gabon à 100–...,Coleus rhodesianum,fr,100.0


## Extract traits

In [17]:
OPENAI_MODEL = "gpt-4"  # or "gpt-3.5-turbo"
OPENAI_API_KEY = "sk-proj----2Z4G9IA"

# "sk-proj-oFJJKfNb_-60HRKQKmlBvFzI--"

In [18]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [19]:
system_message = """You are an expert in botanical trait extraction.

Your task is to extract ONLY the specified trait fields from a botanical species description. Do not guess or infer values — only extract what is explicitly stated. If a trait is not mentioned, set its value to null.

Return your output strictly as a JSON object with exactly the keys listed. Do not add any explanations or extra text.
"""

prompt_fr = """
Extract the following trait fields from the botanical description below. If a field is not mentioned, set it to null.

Trait fields:
"life form", "habitat", "habit", "clonality", "perennial organ", "plant min. height [m]",
"plant max. height [m]", "indumentum", "spinescence", "succulence", "leaf arrangement",
"leaf architecture", "leaf position", "leaf shape", "leaf apex", "leaf base", "leaf margin",
"leaf min. width [cm]", "leaf max. width [cm]", "leaf min. length [cm]", "leaf max. length [cm]",
"bract colour", "bracteole colour", "inflorescence arrangement", "inflorescence orientation",
"flower sex", "flower architecture", "flower merosity", "flower symmetry", "flower shape",
"flower colour", "flower orientation", "petal fusion", "petal colour", "petal min. width [cm]",
"petal max. width [cm]", "calyx colour", "calyx min. length [cm]", "calyx max. length [cm]",
"corolla colour", "corolla tube min. length [cm]", "corolla tube max. length [cm]",
"corolla tube lipped", "labellum colour", "labellum min. width [cm]", "labellum max. width [cm]",
"spur min. length [cm]", "spur max. length [cm]", "reproduction architecture", "reproduction system",
"stamen number", "stamen arrangement", "stamenoid number", "stamen min. length [cm]",
"stamen max. length [cm]", "staminodium", "pollen colour", "carpel/ovary number",
"gynoecium arrangement", "heterostyly", "pollination", "reward", "fruit type", "fruit structure",
"fruit dehiscence", "fruit shape", "fruit colour", "seed colour", "seeds per fruit min",
"seeds per fruit max", "seed min. width [mm]", "seed max. width [mm]", "seed min. length [mm]",
"seed max. length [mm]", "seed min. diameter [mm]", "seed max. diameter [mm]", "dispersal mode",
"ploidy level (2n)", "root min. depth [cm]", "root max. depth [cm]"

Botanical description:
<<<
{description}
>>>

You will be given a French botanical description. Translate the French technical terms faithfully into English. 

"""

prompt_en = """
Extract the following trait fields from the botanical description below. If a field is not mentioned, set it to null.

Trait fields:
"life form", "habitat", "habit", "clonality", "perennial organ", "plant min. height [m]",
"plant max. height [m]", "indumentum", "spinescence", "succulence", "leaf arrangement",
"leaf architecture", "leaf position", "leaf shape", "leaf apex", "leaf base", "leaf margin",
"leaf min. width [cm]", "leaf max. width [cm]", "leaf min. length [cm]", "leaf max. length [cm]",
"bract colour", "bracteole colour", "inflorescence arrangement", "inflorescence orientation",
"flower sex", "flower architecture", "flower merosity", "flower symmetry", "flower shape",
"flower colour", "flower orientation", "petal fusion", "petal colour", "petal min. width [cm]",
"petal max. width [cm]", "calyx colour", "calyx min. length [cm]", "calyx max. length [cm]",
"corolla colour", "corolla tube min. length [cm]", "corolla tube max. length [cm]",
"corolla tube lipped", "labellum colour", "labellum min. width [cm]", "labellum max. width [cm]",
"spur min. length [cm]", "spur max. length [cm]", "reproduction architecture", "reproduction system",
"stamen number", "stamen arrangement", "stamenoid number", "stamen min. length [cm]",
"stamen max. length [cm]", "staminodium", "pollen colour", "carpel/ovary number",
"gynoecium arrangement", "heterostyly", "pollination", "reward", "fruit type", "fruit structure",
"fruit dehiscence", "fruit shape", "fruit colour", "seed colour", "seeds per fruit min",
"seeds per fruit max", "seed min. width [mm]", "seed max. width [mm]", "seed min. length [mm]",
"seed max. length [mm]", "seed min. diameter [mm]", "seed max. diameter [mm]", "dispersal mode",
"ploidy level (2n)", "root min. depth [cm]", "root max. depth [cm]"

Botanical description:
<<<
{description}
>>>

"""

def get_cache_key(prompt, model, temperature):
    data = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature
    }
    data_string = json.dumps(data, sort_keys=True)
    return hashlib.sha256(data_string.encode()).hexdigest()

def call_chatgpt_with_cache(prompt, model="gpt-4-1106-preview", temperature=0):

    cache_key = get_cache_key(prompt, model, temperature)
    cache_file = CACHE_DIR / f"{cache_key}.json"

    if cache_file.exists():
        return json.loads(cache_file.read_text())
        
    response = client.chat.completions.create(
        model=model,
        # model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt.format(description=description)}
        ],
        temperature=temperature,
    )  

    content = response.choices[0].message.content

    try:
        cleaned = content.strip().strip('```json').strip('```').strip()
        data = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(content)
        raise e   
    else:
        cache_file.write_text(json.dumps(data))

    return data

    
    

def extract_traits(description, lang):

    prompt = prompt_en if lang == 'en' else prompt_fr
    prompt = prompt.format(description=description)

    traits = call_chatgpt_with_cache(prompt)
    return traits
    

    


description = """
Herbeannuelle ou pérenne, décombante ou presque dressée, souvent radicante, atteignant 15 cm de hauteur ou 60 cm de longueur ; tige aqueuse, souvent rougeâtre ou violacée, glabre ou pileuse.Feuille: pétiole 1–15 mm, étroitement ailé ; limbe ovale, 1–7 × 0,8–4 cm, base atténuée, sommet aigu ou obtus, marge dentée, glabre ou pileux.Capituleradié, solitaire, hémisphérique, devenant conique, 4–14 × 4–10 mm ; pédoncule 2–7 cm ; involucre 3,5–6 mm de longueur, àbractées bisériées.Fleurons ligulés10 à 15; corolle 2,2–3,3 mm de longueur, jaune à orange, ligule 1,4–2,3 mm de longueur,Fleurons tubulésnombreux ; corolle 1,1–1,9 mm de longueur, jaune à orange clair.Akène1,3–2,4 mm de longueur, glabre ;pappus généralement absent, mais parfois ceux des fleurons externes à quelques soies.
""" 


c = extract_traits(description, lang='fr')        

In [20]:
c

{'life form': 'Herbaceous',
 'habitat': None,
 'habit': 'decumbent or almost erect',
 'clonality': 'often rooting',
 'perennial organ': None,
 'plant min. height [m]': 0.15,
 'plant max. height [m]': None,
 'indumentum': 'glabrous or pilose',
 'spinescence': None,
 'succulence': 'aqueous stem',
 'leaf arrangement': None,
 'leaf architecture': None,
 'leaf position': None,
 'leaf shape': 'oval',
 'leaf apex': 'acute or obtuse',
 'leaf base': 'attenuated',
 'leaf margin': 'toothed',
 'leaf min. width [cm]': 0.8,
 'leaf max. width [cm]': 4,
 'leaf min. length [cm]': 1,
 'leaf max. length [cm]': 7,
 'bract colour': None,
 'bracteole colour': None,
 'inflorescence arrangement': 'radiate, solitary',
 'inflorescence orientation': 'hemispherical, becoming conical',
 'flower sex': None,
 'flower architecture': None,
 'flower merosity': None,
 'flower symmetry': None,
 'flower shape': None,
 'flower colour': None,
 'flower orientation': None,
 'petal fusion': None,
 'petal colour': None,
 'petal

In [31]:
desc_df = results_df[(results_df.section.isin(['general', 'morphology'])) & (results_df.lang.isin(['fr', 'en']))]

# for row in

In [37]:
data = []
x = 0
for i, row in tqdm(desc_df.iterrows(), total=len(desc_df), desc="Extracting traits"):
    try:
        traits = extract_traits(row.text, row.lang)
    except Exception:
        continue

    traits['name'] = row['name']
    traits['lang'] = row.lang
    traits['wfo_taxon_id'] = row.wfo_taxon_id
    data.append(traits)
    x+=1
    if x > 934:
        break

Extracting traits:  38%|███████▏           | 935/2484 [00:00<00:00, 2916.96it/s]


In [38]:
df = pd.DataFrame(data)

In [39]:
df

,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,succulence,...,seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],name,lang,wfo_taxon_id
0,Annuals,None,Culms densely tufted,None,None,0.02,0.25,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Cyperus michelianus,en,wfo-0001302265
1,Tufted annual,None,None,None,None,0.03,0.1,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Cyperus michelianus,en,wfo-0001302265
2,annual herb,None,None,None,None,0.30,0.9,glabrous,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Sphaeranthus talbotii,en,wfo-0000002780
3,herb,None,decumbent or scrambling,often rooting at the nodes,None,0.15,0.6,glabrous or pilose,None,sappy,...,2.4,NaN,NaN,None,None,NaN,NaN,Acmella caulirhiza,en,wfo-0000009712
4,Herbaceous,None,decumbent or almost erect,often rooting,None,0.15,None,glabrous or pilose,None,aqueous stem,...,2.4,NaN,NaN,"pappus generally absent, but sometimes those o...",None,NaN,NaN,Acmella caulirhiza,fr,wfo-0000009712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,Perennial herb,None,"erect, bushy, sprawling, sub-scandent",None,None,0.60,2,thinly pilose to densely tomentose,Spines of modified flowers,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
931,None,None,weed,None,None,0.61,1.22,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
932,Annual or perennial herb,None,"more or less erect, prostrate and sprawling, o...",None,None,0.30,2.5,thinly pilose to densely tomentose,None,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
933,Annual or perennial herb,None,"erect, prostrate and sprawling, or subscandent...",None,None,0.30,2.5,"sericeous, tomentose, subglabrous, pilose",None,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468


In [40]:
col = 'name'
df = df[[col] + [c for c in df.columns if c != col]]

In [41]:
df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. length [mm],seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id
0,Cyperus michelianus,Annuals,None,Culms densely tufted,None,None,0.02,0.25,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0001302265
1,Cyperus michelianus,Tufted annual,None,None,None,None,0.03,0.1,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0001302265
2,Sphaeranthus talbotii,annual herb,None,None,None,None,0.30,0.9,glabrous,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0000002780
3,Acmella caulirhiza,herb,None,decumbent or scrambling,often rooting at the nodes,None,0.15,0.6,glabrous or pilose,None,...,1.3,2.4,NaN,NaN,None,None,NaN,NaN,en,wfo-0000009712
4,Acmella caulirhiza,Herbaceous,None,decumbent or almost erect,often rooting,None,0.15,None,glabrous or pilose,None,...,1.3,2.4,NaN,NaN,"pappus generally absent, but sometimes those o...",None,NaN,NaN,fr,wfo-0000009712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,Pupalia lappacea,Perennial herb,None,"erect, bushy, sprawling, sub-scandent",None,None,0.60,2,thinly pilose to densely tomentose,Spines of modified flowers,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
931,Pupalia lappacea,None,None,weed,None,None,0.61,1.22,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
932,Pupalia lappacea,Annual or perennial herb,None,"more or less erect, prostrate and sprawling, o...",None,None,0.30,2.5,thinly pilose to densely tomentose,None,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
933,Pupalia lappacea,Annual or perennial herb,None,"erect, prostrate and sprawling, or subscandent...",None,None,0.30,2.5,"sericeous, tomentose, subglabrous, pilose",None,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468


In [42]:
df.to_csv('wfo-traits-enfr.csv', index=False)

In [ ]:
df.wfo_taxon_id